In [36]:
from torch.utils.data import DataLoader
from utils import *
from matplotlib import pyplot
import os

In [37]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_default_device(device)
print(device)

cuda


In [38]:
torch.set_default_dtype(torch.float64)
# Okay, we kind of have a way of loading the data
# We need to collect the data and feed it to the transformer model
# Now how do we that ?


# Torch Tensor data !
path = './Datasets/intracardiac_dataset/'
VmTrainData, pECGTrainData, VmTestData, pECGTestData  = fileReaderForActivation(path, 3200)
print('Data loading from files - complete')

VmTrainData = (VmTrainData - torch.min(VmTrainData))/(torch.max(VmTrainData) - torch.min(VmTrainData))
pECGTrainData = (pECGTrainData - torch.min(pECGTrainData))/(torch.max(pECGTrainData) - torch.min(pECGTrainData))

VmTestData = (VmTestData - torch.min(VmTestData))/(torch.max(VmTestData) - torch.min(VmTestData))

pECGTestData = (pECGTestData - torch.min(pECGTestData))/(torch.max(pECGTestData) - torch.min(pECGTestData))

# Let's just transpose the data
pECGTrainData, pECGTestData = pECGTrainData.permute(0,2,1), pECGTestData.permute(0,2,1)

print('Normalization - complete!')


Loading datafiles :  91%|█████████ | 2903/3200 [00:01<00:00, 2462.84it/s]


KeyboardInterrupt: 

In [ ]:
print(pECGTrainData.shape)

torch.Size([2560, 12, 500])


In [41]:
# Define the neural network
from torch import nn
class Network(nn.Module):
    def __init__(self,in_features, out_features):
        super().__init__()
        self.ConvStack = nn.Sequential(
            nn.Conv1d(
                      in_channels = in_features, 
                      out_channels =6, 
                      kernel_size= 5, 
                      stride = 3
                    ) ,
            nn.ReLU(),
            nn.Conv1d(
                      in_channels = 6, 
                      out_channels=3, 
                      kernel_size = 3, 
                      stride =2
                    ),
            nn.ReLU(),
            nn.Conv1d(
                      in_channels = 3,
                      out_channels = 1, 
                      kernel_size = 2, 
                      stride = 1
                    ),
            nn.Sigmoid()
            )
        self.FullyConnectedStack = nn.Sequential(
            nn.Linear(in_features=81, out_features=out_features)
        )

    
    def forward(self, x):
        x = self.ConvStack(x)
        x = self.FullyConnectedStack(x)
        return x.squeeze(1)

In [42]:
# Now let's load the dataloader
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.len = self.X.shape[0]
        
    def __len__(self):
        return self.len

    def __getitem__(self, index):
        return self.X[index], self.y[index]


trainloader = CustomDataset(X = pECGTrainData, y = VmTrainData)
trainloader = DataLoader(trainloader, batch_size=16)


In [43]:
# Define all the parameters for the neural network
import torch.optim as optim


model = Network(in_features=12, out_features=75)
optimizer = optim.Adam(model.parameters(), lr = 1e-3)

lossfn = nn.MSELoss()



In [49]:
EPOCHS = 10
for epoch in range(EPOCHS):
    
    for inp, op in trainloader:
        pred = model(inp.to(device))
        loss  = lossfn(op.to(device), pred)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('EPOCH: {epoch}         Loss: {loss}'.format(epoch = epoch, loss=loss.item()))


EPOCH: 0         Loss: 0.05101811306708155
EPOCH: 1         Loss: 0.05100000634692691
EPOCH: 2         Loss: 0.05097875786507235
EPOCH: 3         Loss: 0.050953358183414225
EPOCH: 4         Loss: 0.05092242431503593
EPOCH: 5         Loss: 0.050884058871685525
EPOCH: 6         Loss: 0.05083570584004094
EPOCH: 7         Loss: 0.050774205037063244
EPOCH: 8         Loss: 0.05069658224202267
EPOCH: 9         Loss: 0.050606723849467676
